In [48]:
from pitchr import data
from pitchr import xml_parser
from pitchr import harmony_maker
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
DATA_PATH = "../dataset/_xml_scores"
score_files = os.listdir(DATA_PATH)

In [3]:
# gets list of all melody and harmony dfs
all_melody_dfs, all_harmony_dfs = xml_parser.get_all_data()
print(len(all_melody_dfs), len(all_harmony_dfs))

210 210


In [4]:
# Converting dfs to numpy arrays
minimum_melody = 999
minimum_harmony = 999
all_melody_np = []
all_harmony_np = []
for df in all_melody_dfs:
    df = df[['Pitch Number', 'Pitch Interval', 'Pitch Predictability']]
    temp_np = df.to_numpy()
    all_melody_np.append(temp_np)
for df in all_harmony_dfs:
    df = df[['Pitch Number', 'Pitch Interval', 'Pitch Predictability']]
    temp_np = df.to_numpy()
    all_harmony_np.append(temp_np)


# normalize harmony sizes
all_harmony_np = xml_parser.prepare_harmony(all_harmony_np)

In [5]:
# convert dataframes to numpy arrays
all_melody_np = np.asarray(all_melody_np)
all_harmony_np = np.asarray(all_harmony_np)

In [47]:
# convert all values to floats
# min pitch number is -33
# max pitch number is 43

for array in all_melody_np:
    for note in array:
        for i in range(0, 3):
            if type(note[i]) == str:
                note[i] = float(-50)
                np.float32
            elif type(note[i]) == int:
                note[i] = float(note[i])
for array in all_harmony_np:
    for note in array:
        for i in range(0, 3):
            if type(note[i]) == str:
                note[i] = float(-50)
            elif type(note[i]) == int:
                note[i] = float(note[i])


                
max_pitch_number = -50
max_interval = -50
max_predictability = -50
for array in all_melody_np:
    for note in array:
        if note[0] > max_pitch_number:
            max_pitch_number = note[0]
        if note[1] > max_interval:
            max_interval = note[1]
        if note[2] > max_predictability:
            max_predictability = note[2]
for array in all_harmony_np:
    for note in array:
        if note[0] > max_pitch_number:
            max_pitch_number = note[0]
        if note[1] > max_interval:
            max_interval = note[1]
        if note[2] > max_predictability:
            max_predictability = note[2]

print(all_melody_np.shape)
print(all_harmony_np.shape)
print(max_pitch_number, max_interval, max_predictability)

(210, 50, 3)
(210, 50, 3)
43.0 33.0 1.0


In [9]:
X = tf.convert_to_tensor(all_melody_np, dtype=tf.float32)
Y = tf.convert_to_tensor(all_harmony_np, dtype=tf.float32)

In [53]:
x_train, x_test, y_train, y_test = train_test_split(all_melody_np, all_harmony_np, test_size=0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.1)
print(x_train.shape, y_train.shape)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
normalized_x_train = x_train/50
normalized_y_train = y_train/50

(151, 50, 3) (151, 50, 3)


In [ ]:
print(all_melody_np.shape)
print(all_harmony_np.shape)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model = Sequential()
# 2 LSTM layers
model.add(LSTM(200, input_shape=(50, 3), return_sequences=True))
model.add(LSTM(200))

# Dense layers
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(3, activation="softmax"))
model.summary()
model.compile(optimizer=optimizer, loss='mse')
print(model.summary())
trained_model = model.fit(normalized_x_train, normalized_y_train, epochs=50, validation_split=0.2, batch_size = 1)

In [22]:
# trained model
summary = model.summary()
print(summary)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 50, 512)           1056768   
_________________________________________________________________
lstm_9 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_8 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 195       
Total params: 3,188,995
Trainable params: 3,188,995
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
# model2. Trying out less layers

optimizer = keras.optimizers.Adam(learning_rate=0.01)
model2 = Sequential()
# 2 LSTM layers
model2.add(LSTM(200, input_shape=(50, 3), return_sequences=True))
model2.add(LSTM(200))

# Dense layers
model2.add(Dense(100, activation="relu"))
model2.add(Dropout(0.3))

# Output Layer
model2.add(Dense(3, activation="softmax"))
model2.summary()
model2.compile(optimizer=optimizer, loss='mse')
print(model2.summary())
trained_model2 = model2.fit(normalized_x_train, normalized_y_train, epochs=50, validation_split=0.2, batch_size = 1)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 50, 200)           163200    
_________________________________________________________________
lstm_29 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dense_28 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 303       
Total params: 504,403
Trainable params: 504,403
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_14"
_________________________________________________________________
Layer (type)            

KeyboardInterrupt: 

In [46]:
model2.summary()


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 50, 200)           163200    
_________________________________________________________________
lstm_25 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dense_24 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 303       
Total params: 504,403
Trainable params: 504,403
Non-trainable params: 0
_________________________________________________________________


In [7]:
model = harmony_maker.build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 512)           1056768   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
Total params: 3,192,050
Trainable params: 3,192,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
# compile model
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])